In [ ]:
import requests
import json
import csv
import time
import boto3
from decimal import Decimal
from requests_aws4auth import AWS4Auth


In [ ]:
restaurants = dict()

In [ ]:
headers = {
    "Authorization" : "Bearer " + input("yelp api key")
}

In [ ]:
def get_restaurants(cuisine, restaurants):
    counter = 0
    params = {
        "term" : cuisine,
        "location" : "New York City",
        "categories" : "restaurants",
        "limit" : 50,
        "offset" : 0
    }
    response = requests.get("https://api.yelp.com/v3/businesses/search", params=params, headers=headers)
    cur_list = json.loads(response.text, parse_float=Decimal)
    while counter + params['offset'] < cur_list['total']:
        unique_counter = 0
        cur_list = json.loads(response.text)
        for res in cur_list['businesses']:
            cur_res = {
                "id" : res["id"],
                "timestamp" : int(time.time() * 1000),
                "cuisine" : cuisine,
                "name" : res["name"],
                "address" : res["location"]['display_address'],
                "latitude" : res["coordinates"]['latitude'],
                "longitude" : res["coordinates"]['longitude'],
                "zip" : res["location"]['zip_code'],
                "review_count" : res["review_count"],
                "rating" : res["rating"],
            }
            if cur_res["id"] not in restaurants:
                unique_counter += 1
            restaurants[cur_res["id"]] = cur_res
        print(unique_counter, ' results for', counter, end=' ')
        counter += len(cur_list['businesses'])
        params['offset'] = counter
        response = requests.get("https://api.yelp.com/v3/businesses/search", params=params, headers=headers)

In [ ]:
for cuisine in ['Chinese', 'Japanese', 'Indian', 'Ethiopian', 'Korean', 'Mediterranean', 'Jamaican', 'Mexican']:
    try:
        get_restaurants(cuisine, restaurants)
    except Exception:
        print("done with ", cuisine)

In [ ]:
sample = dict()
for a in restaurants.values():
    sample = a
    print(sample)
    break

In [ ]:
with open("D:\\Documents\\Documents\\Cloud Computing\\HW1\\restaurants.csv", "w") as res_file:
    row_names = list(sample.keys())
    csv_writer = csv.DictWriter(res_file, row_names)
    csv_writer.writeheader()
    counter = 0
    for val in restaurants.values():
        try:
            csv_writer.writerow(val)
            counter += 1
        except Exception:
            pass
    print(counter, "good records")

In [ ]:
res2 = dict(restaurants)
for res_id,res in restaurants.items():
    cur_res = dict(res)
    cur_res['address'] = res['address']['display_address'][0]
    cur_res['zip'] = res['address']['zip_code']
    cur_res['latitude'] = res['coordinates']['latitude']
    cur_res['longitude'] = res['coordinates']['longitude']
    res2[res_id] = cur_res

In [ ]:
restaurants = dict()

In [ ]:
with open("D:\\Documents\\Documents\\Cloud Computing\\HW1\\restaurants.csv") as res_file:
    reader = csv.DictReader(res_file)
    for row in reader:
        try:
            cur_row = {
                'id' : row['id'],
                'cuisine' : row['cuisine'],
                'name' : row['name'],
                'address' : row['address'],
                'review_count' : int(row['review_count']),
                'rating' : float(row['rating']),
                'zip' : int(row['zip']),
                'latitude' : float(row['latitude']),
                'longitude' : float(row['longitude']),
                'timestamp' : int(time.time() * 1000),
            }
            restaurants[cur_row['id']] = cur_row
        except:
            pass

In [ ]:
dynamodb = boto3.resource('dynamodb',region_name='us-east-1', aws_access_key_id=input('access key:'),
    aws_secret_access_key=input('secret access key: '))
table = dynamodb.Table('yelp-restaurants')

In [ ]:
counter = 0
with table.batch_writer() as batch:
    for res in restaurants.values():
        cur_res = json.loads(json.dumps(res), parse_float=Decimal)
        batch.put_item(
            Item = cur_res
        )

In [ ]:
with open("D:\\Documents\\Documents\\Cloud Computing\\HW1\\restaurants_index.txt", "w") as idx_file:
    for res_id, cur_res in restaurants.items():
        idx_file.write(res_id + ',' + cur_res['cuisine'] + '\n')
    

In [ ]:
region = 'us-east-1'
service = 'es'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)

host = 'https://search-restaurantscuisinedomain-3lysmwsbjfmsnmnc754dmm5x6e.us-east-1.es.amazonaws.com'
index = 'restaurants'
idx_type = '_doc'
url = host + '/' + index + '/' + idx_type + '/'
headers = { "Content-Type": "application/json" }

In [ ]:
for res_id, cur_res in restaurants.items():
    r = requests.put(url + res_id, auth=awsauth, json={'id' : res_id, 'cuisine' : cur_res['cuisine']}, headers = headers)
    print(r.status_code, end=' ')